In [2]:
!pip3 install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/71/46/c6f9179f73b818d5827202ad1c4a94e371a29473b7f043b736b4dab6b8cd/jieba-0.39.zip
    100% |████████████████████████████████| 747kB 11.3MB/s 
  Running setup.py bdist_wheel for jieba ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/c9/c7/63/a9ec0322ccc7c365fd51e475942a82395807186e94f0522243
Successfully built jieba
azure-datalake-store 0.0.19 has requirement msrest~=0.4.5, but you'll have msrest 0.5.0 which is incompatible.


In [3]:
import pycrfsuite
import os,json,logging
logging.basicConfig(level=logging.INFO)

In [7]:
def _word2features(wordlist, i):
    """ 返回特征列表 """
    features = [
        'bias',
        'word:'+wordlist[i][0],
        'word_attr:'+wordlist[i][1]
    ]

    if i > 0:
        features.append('word[-1]:'+wordlist[i-1][0])
        features.append('word[-1]_attr:'+wordlist[i-1][1])
        if i > 1:
            features.append('word[-2]:'+wordlist[i-2][0])
            features.append('word[-2,-1]:'+wordlist[i-2][0]+wordlist[i-1][0])
            features.append('word[-2]_attr:'+wordlist[i-2][1])
    if i < len(wordlist)-1:
        features.append('word[1]:'+wordlist[i+1][0])
        features.append('word[1]_attr:'+wordlist[i+1][1])
        if i < len(wordlist)-2:
            features.append('word[2]:'+wordlist[i+2][0])
            features.append('word[1,2]:'+wordlist[i+1][0]+wordlist[i+2][0])
            features.append('word[2]_attr:'+wordlist[i+2][1])
    return features
def sent2features(sent):
    return [_word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]

In [5]:
def _fragment(inlist, allist, token):
    pos0=0
    for i in range(len(inlist)):
        if inlist[i][1]=='x' and inlist[i][0] in token:
            pos1=i+1
#             logging.info('%i ~ %i'%(pos0, pos1))
            allist.append(inlist[pos0:pos1])
            pos0=pos1
            
    return   

In [6]:
src = "seg_sym"

## test 1
---
将会尝试两种句读.

第一种, 仅将句号都视作断句.

In [82]:
wlist1=[]
for _, _, fnames in os.walk("json/"+src):
    for fname in fnames:
        if '虔门' in fname: continue
        with open("json/"+src+'/'+fname, 'r') as f:
            content=json.load(f)
            logging.info('%s : %i'%(fname, len(content)))
            _fragment(content, wlist1, "。")
            logging.info('ALL LIST : %i'%(len(wlist1)))

INFO:root:seg_sym_宁.json : 19316
INFO:root:ALL LIST : 497
INFO:root:seg_sym_阴阳学堂-夺碑之役.json : 21924
INFO:root:ALL LIST : 899
INFO:root:seg_sym_阴阳学堂外编.json : 8582
INFO:root:ALL LIST : 1193


In [83]:
%time
train_x, train_y = [sent2features(s) for s in wlist1], [sent2labels(s) for s in wlist1]

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 14.8 µs


In [84]:
%time
model = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(train_x, train_y):
    model.append(xseq, yseq)

model.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier
    'feature.possible_states': True,
    # include transitions that are possible, but not observed
    'feature.possible_transitions': True,
    'feature.minfreq': 3
})

CPU times: user 10 µs, sys: 5 µs, total: 15 µs
Wall time: 22.6 µs


开始训练!

In [85]:
%timeit
model.train("./no_qmmf_juhao.crf")

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.8 µs
Feature generation
type: CRF1d
feature.minfreq: 3.000000
feature.possible_states: 1
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 16188
Seconds required: 11.228

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 15747.100353
Feature norm: 1.000000
Error norm: 30505.255778
Active features: 16160
Line search trials: 1
Line search step: 0.000014
Seconds required for this iteration: 0.823

***** Iteration #2 *****
Loss: 3758.706263
Feature norm: 1.796186
Error norm: 4635.358677
Active features: 15210
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.462

***** Iteration #3 *****
Loss: 3197.671797
Feature norm: 1.940814
Error norm: 3010.687662
Active features: 9232
Line sear

要出结果了

In [95]:
%%time
yyxt_f="json/seg_阴阳学堂.json"
yyxt_list=[]
with open(yyxt_f,'r') as f:
    content=json.load(f)
    _fragment(content, yyxt_list, "。")
    logging.info('ALL LIST : %i'%(len(yyxt_list)))

INFO:root:ALL LIST : 13692


CPU times: user 6.64 s, sys: 871 ms, total: 7.51 s
Wall time: 11.9 s


In [97]:
%%time
tagger = pycrfsuite.Tagger()
tagger.open('./no_qmmf_juhao.crf')
test_xs = [sent2features(s) for s in yyxt_list]
test_y=[]
for test_x in test_xs:
    testy = tagger.tag(test_x)
    test_y.append(testy)

CPU times: user 1min 28s, sys: 43.7 s, total: 2min 11s
Wall time: 2min 12s


In [113]:
s=0
ci_l=[]
for i in range(len(test_y)):
    ci=''
    for j in range(len(test_y[i])):
        if test_y[i][j]!='O':
            ci=ci+yyxt_list[i][j][0]
            if test_y[i][j]=='E':
                ci_l.append(ci)
                ci=''
ci_l=list(set(ci_l))
print(ci_l)

['暗羽手', '应天法门总司', '五色笔', '此槌状', '无天法门', '逆刃刀', '氏骆砚', '入色声', '封策镇', '咒牵带咒', '佛家法门场', '利用诀咒', '逆文碑阵', '天法门', '掬仙阵', '入封策', '入碑阵', '衡陵逆文碑', '桃木锥', '两法门', '十二法门阵', '吧骆砚', '伊尹杖', '衡陵逆文碑阵', '左耳耳门穴', '幻兵军队', '暗羽法门', '回应天法门', '幻工技阴阳家', '叫骆砚', '入八公', '擎仙荷', '阵', '时候', '戳戳骆砚', '噬骨虫', '逆文碑', '入梧桐林', '入内崖', '幻草阵', '入山碑', '使节棒', '使节杖']


## 结论

可怜的骆砚。。。
是标注不好的锅

---
## test 2
现已全新加入 《虔门》 手动标注。 再试试. 这次依然是以句号来断句

In [11]:
%%time
wlist2=[]
for _, _, fnames in os.walk("json/"+src):
    for fname in fnames:
#         if '虔门' in fname: continue
        with open("json/"+src+'/'+fname, 'r') as f:
            content=json.load(f)
            logging.info('%s : %i'%(fname, len(content)))
            _fragment(content, wlist2, "。")
            logging.info('ALL LIST : %i'%(len(wlist2)))
train_x2, train_y2 = [sent2features(s) for s in wlist2], [sent2labels(s) for s in wlist2]   

INFO:root:seg_sym_宁.json : 19316
INFO:root:ALL LIST : 497
INFO:root:seg_sym_虔门.json : 47620
INFO:root:ALL LIST : 1752
INFO:root:seg_sym_阴阳学堂外编.json : 8582
INFO:root:ALL LIST : 2046
INFO:root:seg_sym_阴阳学堂-夺碑之役.json : 21924
INFO:root:ALL LIST : 2448


CPU times: user 2.91 s, sys: 558 ms, total: 3.47 s
Wall time: 4.16 s


In [12]:
%%time
trainer = pycrfsuite.Trainer()

for xseq, yseq in zip(train_x2, train_y2):
    trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier
    'feature.possible_states': True,
    # include transitions that are possible, but not observed
    'feature.possible_transitions': True,
    'feature.minfreq': 3
})

CPU times: user 5.54 s, sys: 169 ms, total: 5.71 s
Wall time: 5.73 s


In [14]:
%%time
trainer.train("./juhao.crf")

Feature generation
type: CRF1d
feature.minfreq: 3.000000
feature.possible_states: 1
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 28544
Seconds required: 9.194

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 31016.926077
Feature norm: 1.000000
Error norm: 59848.039746
Active features: 28485
Line search trials: 1
Line search step: 0.000007
Seconds required for this iteration: 0.637

***** Iteration #2 *****
Loss: 7339.454879
Feature norm: 1.804566
Error norm: 8971.316790
Active features: 26844
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.361

***** Iteration #3 *****
Loss: 6258.469560
Feature norm: 1.948436
Error norm: 5843.403140
Active features: 16510
Line search trials: 1
Line search step: 1.000000
Seconds required for thi

In [15]:
%%time
yyxt_f="json/seg_阴阳学堂.json"
yyxt_list=[]
with open(yyxt_f,'r') as f:
    content=json.load(f)
    _fragment(content, yyxt_list, "。")
    logging.info('ALL LIST : %i'%(len(yyxt_list)))

INFO:root:ALL LIST : 13692


CPU times: user 2.98 s, sys: 261 ms, total: 3.24 s
Wall time: 5.8 s


In [17]:
%%time
tagger = pycrfsuite.Tagger()
tagger.open('./juhao.crf')
test_xs = [sent2features(s) for s in yyxt_list]
test_y=[]
for test_x in test_xs:
    testy = tagger.tag(test_x)
    test_y.append(testy)

CPU times: user 32.3 s, sys: 14.9 s, total: 47.2 s
Wall time: 47.5 s


In [18]:
s=0
ci_l=[]
for i in range(len(test_y)):
    ci=''
    for j in range(len(test_y[i])):
        if test_y[i][j]!='O':
            ci=ci+yyxt_list[i][j][0]
            if test_y[i][j]=='E':
                ci_l.append(ci)
                ci=''
ci_l=list(set(ci_l))
print(ci_l)

['使节棒', '暗羽法门', '应天法门总司', '逆刃刀', '透观砚', '脉十二珠', '幻草阵', '戳戳骆砚', '就是九十六珠', '锁魄珠', '阵', '叫骆砚', '玄冥珠', '逆文碑阵', '天法门', '像是108珠', '水银珠', '喜波砚', '双手捻珠', '使节杖', '万道青光', '入山碑', '一共九十六珠', '十方封界', '泪噙珠', '时候把珠', '本体逆文碑', '十二法门阵', '做十二珠', '是不是条虫', '无天法门', '天心骆砚', '褪法门印', '五色笔', '暗羽手', '时候用珠', '幻工技阴阳家', '衡陵逆文碑阵', '桃木锥', '衡陵逆文碑', '空冢', '控制菩提珠', '幻兵军队', '封策镇入封策', '上面水银珠', '阵套阵', '断棒', '血菩提珠', '逆文碑', '入碑阵', '牧羊棒', '掬仙阵', '一棒', '天地阴阳阵', '双龙夺珠', '婆喜虫', '食灵果', '入八公', '佛家法门场', '一串砗磲珠', '封策镇', '阵连阵']


## 结论

不是阵就是珠, 甚至连上次找到的"擎仙荷"都没有!

太令人失望了